In [ ]:
import pandas as pd
import requests 
import re
from bs4 import BeautifulSoup

In [ ]:
!pip install BeautifulSoup

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


#10x10 테스트데이터 크롤링(신상품/ 판매량)

In [ ]:
Category = ['Furniture','Life','Deco','Kitchen','Food','Baby,Kids','FasihonClothing','FasionGoods','Beauty','Jewelry']
Category_num = ['121','120','122','112','119','115','117','116','118','125']               

df = pd.DataFrame()

for j in range(0,2):
    category = Category[j]
    print(category, ' 카테고리 크롤링 시작')
    category_num = Category_num[j]
    key_list = []
    
    #3페이지만 크롤링
    for i in range(1,2):
        
        # 제목 추출 => srm: ne(신상품)/ ds(판매량)  
        url = f'https://www.10x10.co.kr/shopping/category_main.asp?rect=&prvtxt=&rstxt=&extxt=&sflag=n&disp={category_num}&cpg={i}&chkr=False&chke=False&mkr=&sscp=Y&psz=40&srm=ne&iccd=0&styleCd=&attribCd=&icoSize=S&arrCate=&deliType=&minPrc=&maxPrc=&lstDiv=list&ab='
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
        html = requests.get(url, headers=headers)
        title = BeautifulSoup(html.text, 'html.parser')        
        keywords = title.select('p.pdtName.tPad07')        
        
        
        #6개는 행사 상품
        for k in keywords[6:78]:
            keyword = k.text
            key_list.append(keyword)        
        print(i,'페이지 완료')
    
    # [] / () 지우기
    key_list2 = []
      
    for key in key_list :
        if '[' and '(' in key :
            key = re.sub(r'\[[^)]*\]', '', key)
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)        
        elif '[' in key :
            key_re = re.sub(r'\[[^)]*\]', '', key)
            key_list2.append(key_re)
        elif '(' in key :
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)
        elif '★' in key :
            key_re = re.sub(r'\★[^)]*\★', '', key)
            key_list2.append(key_re)
        else :
            key_list2.append(key)
    print('(),[]삭제 완료')    
    
    #형태소 분류
    titles =[]
    for key in key_list2 :
        titles.append(re.compile('[^가-힣]').sub(' ',key))
        # '가'부터 '힣'빼고 나머지 제외(한글모두)
        # ^:뒤에 붙은 것 제외하고 모두
        #정규표현식 or 점프투파이썬 정규표현식 찾아보기

    df_title = pd.DataFrame(titles, columns=['title'])
    df_title['category'] = category
    df = pd.concat([df, df_title], axis = 0, ignore_index=True)
    
    print(f'{category} 추가')
    print('-'*30)

if 'Unnamed: 0' in df.columns :
    df = df.drop(['Unnamed: 0'], axis=1)   
    df = df.dropna(axis=0)
    
# ' ' 스페이스 공백 지우기 
index_list = []
for i in range(len(df)):
    if df['title'][i].isspace() == True :
        index_list.append(i)

df = df.drop(index= index_list, axis = 0)
    
df.to_csv('test_data.csv', encoding='utf-8-sig')    
print('저장완료')


Furniture  카테고리 크롤링 시작
1 페이지 완료
(),[]삭제 완료
Furniture 추가
------------------------------
Life  카테고리 크롤링 시작
1 페이지 완료
(),[]삭제 완료
Life 추가
------------------------------
저장완료


In [ ]:
print(key_list)

['[모던하우스] L 시원한 인견 블루 데이지 소파방석 4인용', '[모던하우스] L 시원한 인견 블루 데이지 방석 50X50', '[모던하우스] L 시원한 인견 블루 데이지 메밀베개', '[모던하우스] L 모달 100% 부드럽고 시원한 7부바지 핑크 플라워 M 4', '[모던하우스] L 모달 100% 부드럽고 시원한 7부바지 그린믹스 꽃마리', '[모던하우스] L 모달 100% 부드럽고 시원한 원피스 여리여리한 잔꽃', '[모던하우스] L 모달 100% 부드럽고 시원한 원피스 그린믹스 꽃마리', '[모던하우스] L 시원한 리플 블루 꽃마리 메밀베개', '[모던하우스] L 한번 삶은 3겹 소창 살림덮개 분홍잔꽃', '[모던하우스] L 한번 삶은 3겹 소창 살림덮개 노란잔꽃', '[모던하우스] L 한번 삶은 3겹 소창 살림덮개 겸용 식탁매트 분홍잔', '[모던하우스] L 시원한 리플 블루 꽃마리 식방석 40X40', '[모던하우스] L 시원한 리플 블루 꽃마리 소파방석 3인용', '[모던하우스] L 시원한 리플 블루 꽃마리 소파방석 4인용', '[모던하우스] L 따뜻한 우리집 크리스마스 패널커튼 90X120', '[모던하우스] L 빈티지 플라워부케 쿠션커버 아이보리 45X45', '[모던하우스] L 레드체크 빈티지 꽃마리자수 티코스터', '[모던하우스] L 레드체크 빈티지 꽃마리 자수 패널커튼 90X120', '[모던하우스] L 빈티지코튼 그린체크 앞치마', '[모던하우스] L 빈티지코튼 그린체크 주방장갑', '[모던하우스] L 모달100% 부드럽고 시원한 7부 여리한 잔꽃플라워 그', '[모던하우스] L 모달100% 부드럽고 시원한 7부 여리한 잔꽃플라워 그', '[소프라움] 토토 모달 차렵이불1+베개커버2 세트 퀸/그린', '[소프라움] 토토 모달 차렵이불1+베개커버1 세트 싱글/그린', '[소프라움] 토토 스프레드 퀸사이즈(그린)', '[소프라움] 토토 스프레드 싱글사이즈(그린)', '[소프라움] 토토 차렵이불 싱글사이즈(그린)', '[소프라움] 토토 누비

#G-market 크롤링

In [ ]:
Category = ['Furniture','Life','Deco','Kitchen','Food','Baby,Kids','FasihonClothing','FasionGoods','Beauty','Jewelry']
Category_url = ['G09&subGroupCode=S041','G08&subGroupCode=S035','G09&subGroupCode=S043&largeCategoryCode=100000093','G08&subGroupCode=S034',
                'G07','G04','G01','G02','G03','G02&subGroupCode=S127&largeCategoryCode=100000027']    

df = pd.DataFrame()

for j in range(0,10):
    category = Category[j]
    print(category, ' 카테고리 크롤링 시작')
    category_url = Category_url[j]
    key_list = []

    # 제목 추출  
    url = f'http://corners.gmarket.co.kr/Bestsellers?viewType=G&groupCode={category_url}'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
    html = requests.get(url, headers=headers)
    title = BeautifulSoup(html.text, 'html.parser')        
    keywords_w = title.select('#gBestWrap > div > div > div > ul > li > a') 
    #print(keywords_w)
    #gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li:nth-child(1) > a

    for k in keywords_w:
        keyword_w = k.text
        key_list.append(keyword_w)     
    print(key_list)
    
    key_list2 = []
        
    for key in key_list :
        if '[' and '(' in key :
            key = re.sub(r'\[[^)]*\]', '', key)
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)        
        elif '[' in key :
            key_re = re.sub(r'\[[^)]*\]', '', key)
            key_list2.append(key_re)
        elif '(' in key :
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)
        elif '★' in key :
            key_re = re.sub(r'\★[^)]*\★', '', key)
            key_list2.append(key_re)
        elif '' == key:
            pass
        else :
            key_list2.append(key)

    print('(),[]삭제 완료')    


    #형태소 분류
    titles =[]
    for key in key_list2 :
        titles.append(re.compile('[^가-힣]').sub(' ',key))
    
    df_title = pd.DataFrame(titles, columns=['title'])
    df_title['category'] = category
    df = pd.concat([df, df_title], axis = 0, ignore_index=True)

    print(f'{category} 추가')
    print('-'*30)

if 'Unnamed: 0' in df.columns :
    df = df.drop(['Unnamed: 0'], axis=1)   
    df = df.dropna(axis=0)
        
# ' ' 스페이스 공백 지우기 
index_list = []
for i in range(len(df)):
    if df['title'][i].isspace() == True :
        index_list.append(i)

df = df.drop(index= index_list, axis = 0)
    
df.to_csv('gmarket.csv', encoding='utf-8-sig')    
print('저장완료')
    

Furniture  카테고리 크롤링 시작
['', '', '', '', '', '[리바트]리바트 꼼므 1200 아이책상+의자 세트 (핑크/블루)', '(특가)국산튼튼한 조립식 렌지대 다이 전자렌지수납장', '[리바트]뉴프렌즈 1200 낮은책장 화이트외', '숨쉬는 침대 매트리스 깔판 높낮이 선택 슈퍼싱글 퀸', '커블체어 와이더 바른자세 의자 jj몰', '[국민가구]리움베이직 1000/1200/1400 컴퓨터/학생용/사무용책상', '[한샘]샘키즈 수납장 1305 특가 모음(20%+5%쿠폰)', '[에이블루]NEW 커블체어 와이더 바른자세 의자 jj', '커블체어 와이더 바른자세 의자 JJ', '[한샘]재크플러스  철제 5단 서랍장', '커블체어 와이더 블랙/바른자세 교정의자', '커블체어 와이더 레드/바른자세 교정의자', '[오엠티]2단 철제 수납 선반 거실 스탠드 옷걸이 행거 OHG-B2', '의자 컴퓨터 사무용 PC방 게이밍 책상 학생 중역 게임', '상나라1인용~6인용접이식밥상 교자상 테이블 좌탁 상', '[상도가구]몬스터캠핑 우드롤테이블 L 감성캠핑 접이식 테이블', '[하포스]전국 빠른배송 단면 침대매트리스 싱글 발통 19.5cm', '퀵퍼 강화유리 컴퓨터책상 게이밍책상 사무용책상', '얼라이브즈 컴퓨터책상/멀티테이블', '[아임홈리빙]LINE 클래식 5단 플라스틱 서랍장', '16피스 사무용 책상 컴퓨터 리클라이너 의자 누베스', '바오 1인용 컴퓨터책상 학생 일자형 좌식책상무료배송', '[상도가구]몬스터캠핑 우드롤테이블 M 감성캠핑 접이식 테이블', '[올리빙]원목 미니 테이블 침대협탁 사이드테이블', '클릭B 메쉬의자/사무용/학생/컴퓨터의자', '[동서가구]인기 LED 1200 정면책상 책장세트 DF911267-1', '[로이첸]베이직 체어 (남성용/여성용) / 교정의자 좌식의자', '[한샘]샘 책장 5단 120cm 시공 (20%+5%쿠폰)', '가성비최강 E0등급 친환경 몸통사용 싱크대 전국택배', '[하포스]전국 빠른배송 단면침대매트리스 슈퍼

#쿠팡 크롤링

In [ ]:
Category = ['Furniture','Life','Deco','Kitchen','Food','Baby,Kids','FasihonClothing','FasionGoods','Beauty','Jewelry']
Category_num = ['184557','115673','407567','185669','194276','221934','498704','498828','176522','499066']

df = pd.DataFrame()

for j in range(0,10):
    category = Category[j]
    print(category, ' 카테고리 크롤링 시작')
    category_num = Category_num[j]
    key_list = []
    
        
    # 제목 추출  
    url = f'https://www.coupang.com/np/categories/{category_num}?'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'}
    html = requests.get(url, headers=headers)
    title = BeautifulSoup(html.text, 'html.parser')        
    keywords = title.select('div.name')        
    

    #6개는 행사 상품
    for k in keywords:
        keyword = k.text
        key_list.append(re.sub('\n    ', '', keyword))  
    #print(key_list[:5])    
    print('페이지 완료')
    # 특수문자 지우기
    # 특수문자 < > 모든 값 지우기
    # re.sub('<[^>]+>', '', s)
    # re.sub('<.*?>', '', s)
    
    key_list2 = []
      
    for key in key_list :
        if '[' and '(' in key :
            key = re.sub(r'\[[^)]*\]', '', key)
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)        
        elif '[' in key :
            key_re = re.sub(r'\[[^)]*\]', '', key)
            key_list2.append(key_re)
        elif '(' in key :
            key_re = re.sub(r'\([^)]*\)', '', key)
            key_list2.append(key_re)
        elif '★' in key :
            key_re = re.sub(r'\★[^)]*\★', '', key)
            key_list2.append(key_re)
        else :
            key_list2.append(key)
    print('(),[]삭제 완료')    
    
    # re.compile(pattern)	패턴 문자열 pattern을 패턴 객체로 컴파일한다
    # 리스트의 형태로 반환이 돼서, sub로 하나의 str으로 만들어준다
    # ^ ->텍스트의 시작지점
    
    titles =[]
    for key in key_list2 :
            titles.append(re.compile('[^가-힣]').sub(' ',key))
    

    df_title = pd.DataFrame(titles, columns=['title'])
    df_title['category'] = category
    df = pd.concat([df, df_title], axis = 0, ignore_index=True)
    
    print(f'{category} 추가')
    print('-'*30)

if 'Unnamed: 0' in df.columns :
    df = df.drop(['Unnamed: 0'], axis=1)   
    df = df.dropna(axis=0)
    
# ' ' 스페이스 공백 지우기 
index_list = []
for i in range(len(df)):
    if df['title'][i].isspace() == True :
        index_list.append(i)

df = df.drop(index= index_list, axis = 0)
    
df.to_csv('project_4/test_data_coupang.csv', encoding='utf-8-sig')    
print('저장완료')
    

#워드클라우드생성

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 58.9MB/s 
     |████████████████████████████████| 450kB 27.9MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
#지마켓 워드클라우드
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

def get_noun(category):
    
    okt=Okt()
    noun = okt.nouns(category)
    for i,v in enumerate(noun):
        if len(v) < 2:
            noun.pop(i)
            
    count = Counter(noun)
    noun_list = count.most_common(100)
    
    return noun_list

def visualize(noun_list):
    
    wc = WordCloud(font_path='Jalnan.ttf',\
              background_color = "white", \
              width = 1000, \
              height=1000, \
              max_words=100, \
              max_font_size=300)
    
    wc.generate_from_frequencies(dict(noun_list))
    wc.to_file('gmarket_WordCloud.png')
    
if __name__=='__main__':
    filename = sys.argv[1]
    
    f= open('gmarket.csv', 'r', encoding='utf-8')
    category = f.read()
    noun_list = get_noun(category)
    visualize(noun_list)


In [ ]:
#쿠팡 워드클라우드
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

def get_noun(category):
    
    okt=Okt()
    noun = okt.nouns(category)
    for i,v in enumerate(noun):
        if len(v) < 2:
            noun.pop(i)
            
    count = Counter(noun)
    noun_list = count.most_common(100)
    
    return noun_list

def visualize(noun_list):
    
    wc = WordCloud(font_path='Jalnan.ttf',\
              background_color = "white", \
              width = 1000, \
              height=1000, \
              max_words=100, \
              max_font_size=300)
    
    wc.generate_from_frequencies(dict(noun_list))
    wc.to_file('Coupang_WordCloud.png')
    
if __name__=='__main__':
    filename = sys.argv[1]
    
    f= open('test_data_coupang.csv', 'r', encoding='utf-8')
    category = f.read()
    noun_list = get_noun(category)
    visualize(noun_list)

In [ ]:
#10x10 워드클라우드
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

def get_noun(category):
    
    okt=Okt()
    noun = okt.nouns(category)
    for i,v in enumerate(noun):
        if len(v) < 2:
            noun.pop(i)
            
    count = Counter(noun)
    noun_list = count.most_common(100)
    
    return noun_list

def visualize(noun_list):
    
    wc = WordCloud(font_path='Jalnan.ttf',\
              background_color = "white", \
              width = 1000, \
              height=1000, \
              max_words=100, \
              max_font_size=300)
    
    wc.generate_from_frequencies(dict(noun_list))
    wc.to_file('10x10_WordCloud.png')
    
if __name__=='__main__':
    filename = sys.argv[1]
    
    f= open('10x10(kor2).csv', 'r', encoding='utf-8')
    category = f.read()
    noun_list = get_noun(category)
    visualize(noun_list)